In [ ]:
vlib = c("tidyverse", "ggpubr", "patchwork", "tidyseurat", "ggrastr",
         "Seurat", "ggsci")
lapply(vlib, require, character.only = TRUE, quietly = TRUE) |> suppressMessages()

base.dir = "."
setwd(base.dir)

saveRDS.gz <- function(object, file, threads=4) {
  con <- pipe(paste0("pigz -p", threads, " > ", file), "wb")
  saveRDS(object, file = con)
  close(con)
}
readRDS.gz <- function(file, threads = parallel::detectCores()) {
  con <- pipe(paste0("pigz -d -c -p", threads, " ", file))
  object <- readRDS(file = con)
  close(con)
  return(object)
}

## R preprocessing steps

In [ ]:
chr_barcode = "barcode_information.txt.gz"
seurat_count = Read10X(data.dir = "seurat_count")
seurat_obj = CreateSeuratObject(counts = seurat_count)
df_md = fread(chr_barcode) %>%
  mutate(cell = paste(file, cell_barcode, sep = "_"),
         sample_time = paste(sample, time, sep = "-")) %>%
  filter(cell %in% rownames(seurat_obj@meta.data)) %>%
  select(cell, sample, time, sample_time, file)
df_md_update = seurat_obj@meta.data %>%
  rownames_to_column("cell") %>%
  left_join(df_md) %>%
  column_to_rownames("cell")

list_seurat_obj = SplitObject(seurat_obj, split.by = "sample_time")

seurat_obj@meta.data$sample_time %>% unique()

# sctransform all samples
list_seurat_obj <- lapply(X = list_seurat_obj, 
                          FUN = SCTransform, 
                          method = "glmGamPoi", 
                          return.only.var.genes = FALSE)

# select integration features and integrate
features <- SelectIntegrationFeatures(object.list = list_seurat_obj, nfeatures = 3000)
seurat_obj_Harmony_SCT <- merge(x = list_seurat_obj[[1]], y = list_seurat_obj[2:length(list_seurat_obj)], merge.data=TRUE)
VariableFeatures(seurat_obj_Harmony_SCT) <- features
seurat_obj_Harmony_SCT = seurat_obj_Harmony_SCT %>%
  RunPCA( verbose = FALSE) 
seurat_obj_Harmony_SCT = seurat_obj_Harmony_SCT %>%
  RunHarmony(assay.use="SCT", group.by.vars = "sample_time") %>%
  RunUMAP(reduction = "harmony", dims = 1:30) %>%
  FindNeighbors(reduction = "harmony", dims = 1:30) %>% 
  FindClusters()

#import_umap

df_umap = read_delim("umap.tsv")
colnames(df_umap) = c("Barcode", "pyUMAP_1", "pyUMAP_2")
df_leiden = read_delim("leiden_res1.tsv")
colnames(df_leiden) = c("Barcode", "leiden")

df_md_update = seurat_obj_Harmony_SCT@meta.data %>%
  rownames_to_column("Barcode") %>%
  left_join(df_leiden) %>%
  column_to_rownames("Barcode")

seurat_obj_Harmony_SCT@meta.data = df_md_update

DimPlot(seurat_obj_Harmony_SCT, group.by = "leiden", pt.size = 0)


mtx_pyUMAP = df_umap %>% column_to_rownames("Barcode") %>% as.matrix()

seurat_obj_Harmony_SCT[["pyUMAP"]] = CreateDimReducObject(
  embeddings = mtx_pyUMAP, key = "pyUMAP_", global = T, assay = "SCT")


DimPlot(seurat_obj_Harmony_SCT, reduction = "pyUMAP", group.by = "leiden", pt.size = 0)
seurat_obj_Harmony_SCT %>% saveRDS("231019_PBMC_LC_split_harmony_integrated.RDS")

In [ ]:
## azimuth pipeline is used for 

library(SeuratDisk)
reference <- LoadH5Seurat("pbmc_multimodal.h5seurat") # from azimuth reference
DimPlot(object = reference, reduction = "wnn.umap", group.by = "celltype.l2", label = TRUE, label.size = 3, repel = TRUE,
        pt.size = 0) + NoLegend()
anchors <- FindTransferAnchors(
  reference = reference,
  query = seurat_obj_Harmony_SCT,
  normalization.method = "SCT",
  reference.reduction = "spca",
  dims = 1:50
)
seurat_obj_projection <- MapQuery(
  anchorset = anchors,
  query = seurat_obj_Harmony_SCT,
  reference = reference,
  refdata = list(
    celltype.l1 = "celltype.l1",
    celltype.l2 = "celltype.l2",
    predicted_ADT = "ADT"
  ),
  reference.reduction = "spca", 
  reduction.model = "wnn.umap"
)
rm(seurat_obj_Harmony_SCT); gc()
saveRDS(seurat_obj_projection, file = "231019_PBMC_LC_split_pyintegrated_azimuth.RDS")

## visualizations

In [ ]:
seurat_obj = readRDS.gz("231019_PBMC_LC_split_harmony_integrated.RDS")

In [ ]:
c_color_anno_l1 = pal_flatui(alpha = 0.4)(8)
c_colors = c("#9cc964","#1077f3", "#5ba8f7", "#e83326", "#f98517", 
             "#fbaa5d", "#c85b00", "#ff7d67", "#d3ba59", "#eee8b6",
             "#997600", "#c6a000", "#175182","#48846c", "#ac0000", 
             "#ee74ee", "#970098", "#cc34cd", "#7018d3", "#9b54f3", 
             "#cab1e5", "#903900", "#0050ae", "#008c5c","grey50", "#83ca9c")

## figure 1D-E

In [ ]:
dbl_xmin = -10
dbl_ymin = -11
p_anno_l1 = DimPlot(seurat_obj,
  reduction = "pyUMAP",
  group.by = "anno_l1",
  cols = c_color_anno_l1,
  raster.dpi = c(4096, 4096), pt.size = 4, alpha = 0.4
) &
  guides(color = guide_legend(
    ncol = 1,
    override.aes = list(shape = 15, size = 5)
  )) &
  annotate("segment", x = dbl_xmin, y = dbl_ymin, xend = dbl_xmin, yend = dbl_ymin + 5,
    arrow = arrow(length = unit(.25, "cm"), angle = 20, type = "closed")
  ) &
  annotate("segment", x = dbl_xmin, y = dbl_ymin, xend = dbl_xmin + 5, yend = dbl_ymin,
    arrow = arrow(length = unit(.25, "cm"), angle = 20, type = "closed")
  ) &
  ggtitle("anno_l1") &
  theme(
    plot.title = element_text(size = 20, hjust = 0),
    axis.title.x = element_text(hjust = 0.1, vjust = 0, size = 18),
    axis.title.y = element_text(hjust = 0.1, vjust = 1, size = 18),
    axis.ticks.x = element_blank(),
    axis.ticks.y = element_blank(),
    axis.line = element_blank(),
    axis.text.x = element_blank(),
    axis.text.y = element_blank(),
    legend.text = element_text(size = 12),
    legend.key.height = unit(0.25, "null"),
    legend.key.width = unit(0.5, "cm"),
    legend.position = "right"
  ) &
  xlab("UMAP_1") &
  ylab("UMAP_2")

options(repr.plot.width = 8, repr.plot.height = 7, repr.plot.res = 300)
p_anno_l1 %>% ggsave("figure_prep/pDimPlot_annol1.pdf", ., dpi = 300, height = 7, width = 8)

In [ ]:

pComp_annol1 = seurat_obj@meta.data %>%
  mutate(time = factor(time, levels = c("base", "1st")),
         Response = factor(Response, levels = c("PR", "PD"))) %>%
  ggplot(aes(x = time, fill = anno_l1)) + 
    geom_bar(position  = "fill", alpha = 0.8) + 
    scale_fill_flatui() +
    facet_wrap(~Response, scales = "free_x", )+
    xlab("Before/After treatment")+
    ylab("Proportion")+
    theme_pubr(legend = "right") +
  theme(
    plot.title = element_text(size = 20, hjust = 0),
    axis.title.x = element_text(hjust = 0.5, size = 18),
    axis.title.y = element_text(hjust = 0.5, size = 18),
    axis.text.x =  element_text(size = 16),
    axis.text.y =  element_text(size = 16),
    axis.line = element_blank(), 
    strip.background = element_rect(fill= NA),
    strip.text = element_text(size = 16),
    legend.text = element_text(size = 16),
    legend.key.size = unit(2, 'mm'),
    legend.position = "right",
    panel.border = element_rect(linewidth = 1, color = "black", fill = NA)
  )+ guides(colour = guide_legend(ncol = 1))

pComp_annol1 %>% ggsave("figure_prep/pComp_annol1.pdf", ., dpi = 300, height = 1800, width = 1500, units = "px")



In [ ]:
features= c("CD74", "HLA-DPB1", 'CD79A',
            'IL7R', 'MAL',
            'CD8B', 'CD8A', 
            'NKG7', 
            'GZMK', 'KLRB1',
            'TYROBP',
            'CTSS',
            "PPBP")

pDotPlot = seurat_obj %>% DotPlot(features = features, group.by = "anno_l1")

pDotPlot_updated = pDotPlot$data %>% 
  mutate(id = ifelse(id == "Other_T", "Other T", as.character(id))) %>%
  rename(`Average Expression` = avg.exp.scaled, 
         `% Expressed` = pct.exp) %>%
  mutate(id = factor(id, levels = c("B", "CD4", "CD8", "Other T", "NK", "Mono", "DC", "Other"))) %>%
  ggplot(aes(x = id, y = features.plot, color = `Average Expression`, size = `% Expressed`)) + geom_point() +
  theme_pubr(x.text.angle = 90) + 
  scale_color_gradient2(low = "#624848", mid = "#cad8e6", high = "#13395d")+
  xlab('Annotated clusters') + ylab('Marker genes') +
  theme(axis.line = element_blank(), 
        axis.text.x = element_text(hjust = 1, vjust = 0.5),
        strip.background = element_rect(fill= NA),
        strip.text = element_text(size = 16),
        legend.text = element_text(size = 12),
        legend.key.size = unit(5, 'mm'),
        legend.ticks = element_blank(),
        legend.position = "bottom",
        panel.background = element_blank(),
        panel.border = element_rect(linewidth = 1, color = "black", fill = NA),
        panel.grid.major  = element_line(color = 'grey95'))

pDotPlot_updated %>% ggsave("figure_prep/pDotPlot_markers_annol1.pdf", ., dpi = 300, height = 1800, width = 1000, units = "px")

## supplementary figure 1A

In [ ]:
dbl_xmin = -10
dbl_ymin = -11
p_anno_l2 = DimPlot(seurat_obj,
                      reduction = "pyUMAP",
                      group.by = "anno_l2",
                      #group.by = "anno_c1",
                      #cols = pal_flatui(alpha = 0.4)(8),
                      cols = c(c_colors, pal_simpsons()(3)),
                      #cols = pal_igv(alpha = 0.4)(29),
                      raster.dpi = c(4096, 4096), pt.size = 4, alpha = 0.4,
                      #label = TRUE
                      ) &
  guides(color = guide_legend(
    ncol = 1,
    override.aes = list(shape = 15, size = 5)
  )) &
  annotate("segment", x = dbl_xmin, y = dbl_ymin, xend = dbl_xmin, yend = dbl_ymin + 5,
    arrow = arrow(length = unit(.25, "cm"), angle = 20, type = "closed")
  ) &
  annotate("segment", x = dbl_xmin, y = dbl_ymin, xend = dbl_xmin + 5, yend = dbl_ymin,
    arrow = arrow(length = unit(.25, "cm"), angle = 20, type = "closed")
  ) &
  ggtitle("anno_l2") &
  theme(
    plot.title = element_text(size = 20, hjust = 0),
    axis.title.x = element_text(hjust = 0.1, vjust = 0, size = 18),
    axis.title.y = element_text(hjust = 0.1, vjust = 1, size = 18),
    axis.ticks.x = element_blank(),
    axis.ticks.y = element_blank(),
    axis.line = element_blank(),
    axis.text.x = element_blank(),
    axis.text.y = element_blank(),
    legend.text = element_text(size = 12),
    legend.key.height = unit(1, "null"),
    legend.key.width = unit(0.5, "cm"),
    legend.position = "right"
  ) &
  xlab("UMAP_1") &
  ylab("UMAP_2")

options(repr.plot.width = 8, repr.plot.height = 7, repr.plot.res = 300)
p_anno_l2 %>% ggsave("figure_prep/pDimPlot_annol2.pdf", ., dpi = 300, height = 7, width = 8)

## supplementary figure 1B

In [ ]:
pComp_annol2 = seurat_obj@meta.data %>%
  mutate(time = factor(time, levels = c("base", "1st")),
         Response = factor(Response, levels = c("Responder", "Non-responder"))) %>%
  ggplot(aes(x = time, fill = anno_l2)) + 
  geom_bar(position  = "fill", alpha = 0.8) + 
  scale_fill_manual(values = c(c_colors, pal_simpsons()(3)))+
  facet_wrap(~Response, scales = "free_x", )+
  xlab("Before/After treatment")+
  ylab("Proportion")+
  theme_pubr(legend = "right") +
  theme(
    plot.title = element_text(size = 20, hjust = 0),
    axis.title.x = element_text(hjust = 0.5, size = 18),
    axis.title.y = element_text(hjust = 0.5, size = 18),
    axis.text.x =  element_text(size = 16),
    axis.text.y =  element_text(size = 16),
    axis.line = element_blank(), 
    strip.background = element_rect(fill= NA),
    strip.text = element_text(size = 16),
    legend.text = element_text(size = 12),
    legend.key.size = unit(2, 'mm'),
    legend.position = "right",
    panel.border = element_rect(linewidth = 1, color = "black", fill = NA)
  )+ guides(fill= guide_legend(ncol = 1))
pComp_annol2  %>% ggsave("figure_prep/pComp_annol2.png", ., dpi = 300, height = 1800, width = 1500, units = "px")


## supplementary figure 1C

In [ ]:
c_clusters = c("Naive_B",
  "AFF3_B",
  "Memory_B",
  "Plasmablast",
  "CD4_Naive",
  "CD4_CTL",
  "CD4_TEM",
  "CD4_TCM",
  "CD8_Naive",
  "CD8_TCM",
  "CD8_TEM",
  "Treg",
  "MAIT",
  "gdT",
  "NKT",
  "dnT",
  "Proliferating_T",
  "Mito-enriched",
  "Stress-related",
  "NK_CD56Dim",
  "NK_CD56High",
  "NK_Other",
  "cDC",
  "pDC",
  "CD14_Mono",
  "CD16_Mono",
  "Erythrocyte",
  "Platelet",
  "doublet"
)


c_markers = c("MS4A1", "BANK1", "ITM2C", # B
           "MZB1", # Plasmablast
           "CD4", "GZMH", # CD4 CTL
           "IL7R", # CD4 Naive
           "AQP3", # CD4 TCM
           "CCL5",  # CD4 TEM
           "CD8B",  "CD3D", # CD8 Naive, #CD8 Proliferating
           "NELL2", # CD8 TCM
           "MKI67", # Proliferating
           "KLRB1", "GZMK", # MAIT
           "FOXP3",  # Treg
           "NUCB2", "FXYD2", # dnT
           "TRGV9", "KLRD1", #gdT
           "XCL1", "KLRC1", # NK_high
           "NKG7", "GZMB", "SPON2", # NK_low
           "CLSPN", "TRDC", # NK_Prolifer
           "S100A8", "S100A9", "LYZ", # CD14 Mono
           "LST1", "FCGR3A", "MS4A7", # CD16 Mono
           "FCER1A", "CLEC10A", # DC
           "AHSP", "HBM", # Eryth
           "MT-CO3", "JUNB", #HSPC
           "PPBP", "TUBB1" # Platelet
)
seurat_obj = seurat_obj %>%
  mutate(anno_l2_factor = factor(anno_l2, levels = rev(c_clusters)))

options(repr.plot.width = 13, repr.plot.height = 10, repr.plot.res = 100)
DotPlot(seurat_obj, group.by = "anno_l2_factor", c_markers) &
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1))

ggsave("figure_prep/pDotPlot_markers_annol2.pdf", width = 13, height = 10)